In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 시작하기
kaggle에서 kaggle-survey-2020 data를 받은 후 살펴보기로 합시다. 파일은 하나밖에 없지만, 여러 파일을 불러와 concatenate하는 코드가 더 확장성이 있을 것 같아

glob을 import하고 total이라는 DataFrame 객체에 데이터를 저장합니다. 

In [ ]:
import glob

path = r'/kaggle/input/kaggle-survey-2020' 
all_files = glob.glob(path + r"/*.csv")

total = pd.DataFrame()


for file_name in all_files:

  temp = pd.read_csv(file_name, sep=',', dtype='str')

  total = pd.concat([total, temp], axis=0)

print(total.info())
total.head()


약 20000개의 row와 355개의 column이 있는 것을 확인했습니다. column자체는 문제 번호를 의미하고 1번째 행(idx=0)에 질문이 쓰여 있습니다.

이 질문들을 살펴보겠습니다. 질문이 길고, 그 수가 많아서 옵션을 세부지정해서 출력합니다.

신상정보에 대한 질문과 주로 사용하는 Framework, 향후 익숙해지고 싶은 툴 등 총 39개의 질문으로 이루어져 있습니다. 다중선택을 분리해 놓아 column의 수가 많이 늘어났습니다

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', 20000, 'display.expand_frame_repr', True):  # more options can be specified also
    print(total.iloc[0,:])

예를 들어, 39번 질문에 대한 답안은 12가지 클래스로 분류되어 있는데, 사용자가 어떠한 매체를 선호한다고 답변하면 해당 클래스에 Nonzero인 문자열이 생기는 방식입니다.

이를 통해서 kaggle 사용자들이 데이터 사이언스 관련 리포트를 받아볼 때 가장 선호하는 매체를 알 수 있습니다.

흥미로운 질문-답이므로 39번 문제에 대해 EDA를 진행하겠습니다.



그러나 이대로라면 Part_4, Part_6등에 대한 답변이 무엇이었는지 알기가 힘듭니다.

따라서, 내장 함수 map을 통해 질문-답 인덱스와 실제 질문-답을 연결하겠습니다

In [ ]:
df_q39 = total.loc[:,total.columns.str.contains('Q39')].reset_index(drop=True)
df_q39_sum = df_q39.notna().sum(axis=0)
df_q39_sum.sort_values(ascending=False)

In [ ]:
qna = total.iloc[0,:] #실제 질문-답입니다
col = total.columns # 질문-답 인덱스입니다. 현재 column입니다.
col_to_qna = col.map(qna)
#print(col_to_qna)

df_q39 = total.loc[:,total.columns.str.contains('Q39')].reset_index(drop=True) 
q39_qna = df_q39.columns.map(qna) 
q39_qna = [i[121:].split(' ')[0] for i in q39_qna] # 질답에서 공통되는 부분을 지우고 한 단어만 추출합니다

df_q39_sum = df_q39.notna().sum(axis=0)
df_q39_sum.index = q39_qna
df_q39_sum_sorted = df_q39_sum.sort_values(ascending=False)
df_q39_sum_sorted

# 시각화
이제 순위를 얻었으니 그 선호 비율을 구하고, 시각화해보겠습니다.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
print('***** 캐글러들이 선호하는 데이터 사이언스 정보를 얻는 매체 (%) *****')
print(df_q39_sum_sorted/len(total) * 100)
fig, ax = plt.subplots(figsize=(8,15))
colors = sns.color_palette('hls',len(df_q39_sum_sorted.index))
ax = plt.bar(df_q39_sum_sorted.index, df_q39_sum_sorted.values/len(total) * 100, color=colors)
plt.title(' Favorite Media Sources That Report on Data Science Topics(%) ')
plt.xticks(rotation=24)
plt.grid()
plt.show()